In [1]:
import keras
from keras.layers import *
from keras.preprocessing import *
from keras.optimizers import *

import pandas as pd
import numpy as np

from sklearn.metrics import *

from RocAucEvaluation import RocAucEvaluation

from sklearn.base import BaseEstimator
from keras.models import Model
from keras.regularizers import l2 as l2_reg
import itertools

import csv

import tensorflow as tf

/home/mohsin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

#train = pd.read_csv("../input/train_sample.csv",  dtype=dtypes)
#train.head()

In [3]:
test = pd.read_csv("../input/test_noval.csv", dtype=dtypes)
test.head()

,ip,app,device,os,channel,weekofday,hourofday,is_attributed,ip_device_os


In [10]:
MAX_IP_DEVICE_OS = 3669418
MAX_IP = 364779
MAX_APP = 769
MAX_DEVICE = 4227
MAX_OS = 956
MAX_CHANNEL = 500
NUM_LAYERS = 2
def build_model():
    ip_device_os = Input(shape=(1,))
    ip_device_os_embed = Embedding(MAX_IP_DEVICE_OS, 50)(ip_device_os)
    
    ip = Input(shape=(1,))
    ip_embed = Embedding(MAX_IP, 50)(ip)
    
    app = Input(shape=(1,))
    app_embed = Embedding(MAX_APP, 20)(app)
    
    device = Input(shape=(1,))
    device_embed = Embedding(MAX_DEVICE, 20)(device)
    
    os = Input(shape=(1,))
    os_embed = Embedding(MAX_OS, 15)(os)
    
    channel = Input(shape=(1,))
    channel_embed = Embedding(MAX_CHANNEL, 10)(channel)
    
    hourofday = Input(shape=(1,))
    hourofday_embed = Embedding(24, 10)(hourofday)
    inputs = [ip_device_os, ip, app, device, os, channel, hourofday]
    
    ip_embed2 = Embedding(MAX_IP, 10)(ip)
    app_embed2 = Embedding(MAX_APP, 10)(app)
    
    ip_embed3 = Embedding(MAX_IP, 10)(ip)
    hourofday_embed2 = Embedding(24, 10)(hourofday)
    
    channel_embed2 = Embedding(MAX_CHANNEL, 10)(channel)
    app_embed3 = Embedding(MAX_APP, 10)(app)
  
    channel_embed3 = Embedding(MAX_CHANNEL, 10)(channel)
    device_embed2 = Embedding(MAX_APP, 10)(app)
    
    
    ip_app_embed = Dot(2)([ip_embed2, app_embed2])
    ip_hour_embed = Dot(2)([ip_embed3, hourofday_embed2])
    app_channel_embed = Dot(2)([channel_embed2, app_embed3])
    device_channel_embed = Dot(2)([device_embed2, channel_embed3])
    
    
    
    x = concatenate([Flatten()(ip_device_os_embed),
                     Flatten()(ip_embed), 
                     Flatten()(app_embed), 
                     Flatten()(device_embed),
                     Flatten()(os_embed), 
                     Flatten()(channel_embed),
                     Flatten()(hourofday_embed),
                    Flatten()(ip_app_embed),
                    Flatten()(ip_hour_embed),
                    Flatten()(app_channel_embed),
                    Flatten()(device_channel_embed)])
    
    for _ in range(NUM_LAYERS+1):
        #x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        x = Dense(150)(x)
        x = PReLU()(x)
    
    #x = Dropout(0.1)(x)  
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=out)
    
    #adam = TFOptimizer(tf.train.FtrlOptimizer(0.02))
    adam = Adam(lr=0.001, decay=0.00001)
    model.compile(optimizer=adam, loss='binary_crossentropy')
    
    return model
    

In [8]:
def gen(file_name, batchsz = 10000):
    csvfile = open(file_name)
    reader = csv.reader(csvfile)
    cols = next(reader)
    #print(cols)
    batchCount = 0
    #inputs = []
    ips = []
    apps = []
    devices = []
    oss = []
    channels = []
    hours = []
    targets = []
    ip_device_os = []
    while True:
        try:
            line = next(reader)
        except:
            csvfile.seek(0)
            #reader = csv.reader(csvfile)
            cols = next(reader)
            line = next(reader)
            
        #temp_image = cv2.imread(line[]) # line[1] is path to image
        #measurement = line[0:6] # steering angle
        ips.append(line[0])
        apps.append(line[1])
        devices.append(line[2])
        oss.append(line[3])
        channels.append(line[4])
        hours.append(line[6])
        #inputs.append(line[0:5])
        targets.append(line[7])
        ip_device_os.append(line[8])
        batchCount += 1
        if batchCount >= batchsz:
            batchCount = 0
            X = [np.array(ip_device_os),np.array(ips), np.array(apps), np.array(devices), np.array(oss), np.array(channels),
                np.array(hours)]
            y = np.array(targets)
            ips = []
            apps = []
            devices = []
            oss = []
            channels = []
            hours = []
            targets = []
            ip_device_os = []
            yield X, y
    

In [8]:
##tmp = gen("../input/train_sample.csv")
#next(tmp)

In [6]:
val_df = pd.read_csv("../input/validation.csv", dtype=dtypes)
val_df.head()

,ip,app,device,os,channel,weekofday,hourofday,is_attributed,ip_device_os
0,38877,2,1,19,477,9,4,0,76740
1,104271,14,1,18,489,9,4,0,336274
2,92922,3,1,13,280,9,4,0,2693814
3,6505,3,1,13,19,9,4,0,175211
4,19964,9,1,13,334,9,4,0,162761


In [7]:
X_val = [val_df['ip_device_os'], val_df['ip'], val_df['app'], val_df['device'] , val_df['os'], val_df['channel'], val_df['hourofday']]
y_val = val_df['is_attributed']

In [13]:
#https://github.com/keras-team/keras/issues/7729
#def generate_arrays_from_file(path):
#    while 1:
#        f = open(path)
#        for line in f:
#            # create Numpy arrays of input data
#            # and labels, from each line in the file
#            x, y = process_line(line)
#            yield (x, y)
#    f.close()
roc_auc = RocAucEvaluation(validation_data=(X_val, y_val))
model = build_model()
print(model.summary())
model.fit_generator(gen('../input/train_noval.csv'),
        steps_per_epoch=1740, epochs=20, validation_data=(X_val, y_val), callbacks=[roc_auc])


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_27 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_28 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_22 (

Epoch 1/20
1739/1740 [============================>.] - ETA: 0s - loss: 0.0121
 ROC-AUC - epoch: 1 - score: 0.952324 

1740/1740 [==============================] - 158s 91ms/step - loss: 0.0121 - val_loss: 0.0095
Epoch 2/20
1739/1740 [============================>.] - ETA: 0s - loss: 0.0077
 ROC-AUC - epoch: 2 - score: 0.952621 

1740/1740 [==============================] - 159s 92ms/step - loss: 0.0077 - val_loss: 0.0092
Epoch 3/20
1739/1740 [============================>.] - ETA: 0s - loss: 0.0078
 ROC-AUC - epoch: 3 - score: 0.952172 

1740/1740 [==============================] - 159s 91ms/step - loss: 0.0078 - val_loss: 0.0089
Epoch 4/20
1739/1740 [============================>.] - ETA: 0s - loss: 0.0062
 ROC-AUC - epoch: 4 - score: 0.958451 

1740/1740 [==============================] - 159s 91ms/step - loss: 0.0062 - val_loss: 0.0092
Epoch 5/20
1739/1740 [============================>.] - ETA: 0s - loss: 0.0073
 ROC-AUC - epoch: 5 - score: 0.960548 

1740/1740 [==================

In [12]:
#Let's debug network


In [80]:
list(zip(tmp.T))

[(array([1, 2, 1]),), (array([2, 1, 1]),)]